<a href="https://colab.research.google.com/github/JairParra/ANOVA_with_R/blob/master/Text_generation_with_Recurrent_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -*- coding: utf-8 -*-
"""
Created on Wed Jul 17 23:32:57 2019

@author: jairp
"""

### *** Text Generation with a Recurrent NN *** ### 

In [1]:
### *** Text Generation with a Recurrent NN *** ### 


from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.compat.v1.enable_eager_execution()
eager = tf.executing_eagerly() 
print(eager)

import numpy as np
import os
import time


True


In [2]:
## Download the Shakespear dataset

# https://raw.githubusercontent.com/mxw/grmr/master/src/finaltests/bible.txt
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# path_to_file = tf.keras.utils.get_file('bible.txt', 'https://raw.githubusercontent.com/mxw/grmr/master/src/finaltests/bible.txt')
path_to_file = tf.keras.utils.get_file('trump.txt', 'https://raw.githubusercontent.com/ryanmcdermott/trump-speeches/master/speeches.txt')
# path_to_file = tf.keras.utils.get_file('pishing.txt', 'https://raw.githubusercontent.com/JairParra/FakePisher/master/data_raw/fradulent_emails.txt?token=AF7ALARG2LDBSEGITAV4YWC5HIT52')



tf.test.gpu_device_name()


'/device:GPU:0'

In [3]:
## Read the Data


# Read, then decode for py2 compat
# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')


# length of text is the number of characters in it 
print ('Length of text: {} characters'.format(len(text)))

# Take a look at the first 250 cahracters in text
print(text[:500])

# The unique characters in the file 
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

Length of text: 903947 characters
﻿SPEECH 1


...Thank you so much.  That's so nice.  Isn't he a great guy.  He doesn't get a fair press; he doesn't get it.  It's just not fair.  And I have to tell you I'm here, and very strongly here, because I have great respect for Steve King and have great respect likewise for Citizens United, David and everybody, and tremendous resect for the Tea Party.  Also, also the people of Iowa.  They have something in common.  Hard-working people.  They want to work, they want to make the country 
94 unique characters


In [4]:
### Process the text ### 

## Vectorize the text  

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)


# convert the text to int array 
text_as_int = np.array([char2idx[c] for c in text] )

# NOw we have an integer representation for each characters. Notice that 
# we mapped the character as indexes from 0 to len(unique) 

print('{')
for char,_ in zip(char2idx, range(40)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

    
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(
    repr(text[:13]), text_as_int[:13]))


{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  ',' :  11,
  '-' :  12,
  '.' :  13,
  '/' :  14,
  '0' :  15,
  '1' :  16,
  '2' :  17,
  '3' :  18,
  '4' :  19,
  '5' :  20,
  '6' :  21,
  '7' :  22,
  '8' :  23,
  '9' :  24,
  ':' :  25,
  ';' :  26,
  '=' :  27,
  '?' :  28,
  '@' :  29,
  'A' :  30,
  'B' :  31,
  'C' :  32,
  'D' :  33,
  'E' :  34,
  'F' :  35,
  'G' :  36,
  'H' :  37,
  'I' :  38,
  'J' :  39,
  ...
}
'\ufeffSPEECH 1\r\n\r\n' ---- characters mapped to int ---- > [93 48 45 34 34 32 37  2 16  1  0  1  0]


In [0]:
## The prediction task 

# Given a character, or a sequence of cahracters, what is the most 
# probable next character? This is a the task we are training the model to perform. 
# The input to the model will be a sequence of characters, and we train the model 
# to predict the output. 

# Since RNNs maintain an internal state that depends on the previously 
# seen elements, given all the characters computed unil this moment, 
# what is the next characters? 



# ***************************************************************


### Create trianing examples and targets ### 


# Next divide the text into example sequences. 
# Each input sequence will contain seq_length characters from the text. 
# For each input seq, the corresponding targets contain the same length 
#   of except shifted on echaracter to the right. 
# So break the text into chunks of seq_length+1. For example, 
#   say se_length is 4 and out text is "Hello". The input seuaence 
#   will be "Hell" and the target "ello". 

# To do this, first use the  tf.data.Dataset.from_tensor_slices function to 
#   convert the text vector into a stream of character indices.



In [5]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(20):
  print(idx2char[i.numpy()])

﻿
S
P
E
E
C
H
 
1









.
.
.
T
h


In [6]:
# The *batch* method lets us easily conver these individuals characters 
#    to sequences of the desired size. 
    
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"\ufeffSPEECH 1\r\n\r\n\r\n...Thank you so much.  That's so nice.  Isn't he a great guy.  He doesn't get a fair p"
"ress; he doesn't get it.  It's just not fair.  And I have to tell you I'm here, and very strongly her"
'e, because I have great respect for Steve King and have great respect likewise for Citizens United, D'
'avid and everybody, and tremendous resect for the Tea Party.  Also, also the people of Iowa.  They ha'
've something in common.  Hard-working people.  They want to work, they want to make the country great'


In [7]:
# For each sequence, duplicate and shift it to form the 
# input and target text by using the map method to apply 
#  a simple function to each batch: 

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Print the first examples and target values 

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
  

Input data:  "\ufeffSPEECH 1\r\n\r\n\r\n...Thank you so much.  That's so nice.  Isn't he a great guy.  He doesn't get a fair "
Target data: "SPEECH 1\r\n\r\n\r\n...Thank you so much.  That's so nice.  Isn't he a great guy.  He doesn't get a fair p"


In [8]:
# Each of the index of thes evectors are preocessed as one time step. 
# For the input at time step 0, the model received the index for 
# "F" and tris to predict the index for "i" as the next character. 
# At the next timestep, it does the same thing, but the RNN 
# considers the previous step context in addition to the 
# current input character. 
  

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))


Step    0
  input: 93 ('\ufeff')
  expected output: 48 ('S')
Step    1
  input: 48 ('S')
  expected output: 45 ('P')
Step    2
  input: 45 ('P')
  expected output: 34 ('E')
Step    3
  input: 34 ('E')
  expected output: 34 ('E')
Step    4
  input: 34 ('E')
  expected output: 32 ('C')


In [9]:
# ****************************************************************8
    
### Create training batches ### 
    
# We used *tf.data* to split the text into manageable sequences. 
# But before fedding this data into the model, we need to tshuffle 
# the data and pack it into batches. 
    
# Batch size
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [10]:
# *******************************************************************

### Build The Model ### 

# use tf.keras.Sequential to define the model. For this simple example, three 
# layers are used: 

    # tf.keras.layers.Embedding: 
        # The input layer. A trainable lookup table that will
        # map the numbers of each character vector with 
        # *embedding_dim* dimensions. 
    # td.keras.layers.GRU: 
        # A type of RNN with size units=rnn_units 
        # (can also use a LSTM layer here) 
    # tf.keras.layers.Dense: 
        # The ouput layer, with *vocab_size* outputs. 
        

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


# Next define a function to build the model 
# Use CuDNNGRU if runnin gon GPU 

print("Is GPU available: ", tf.test.is_gpu_available())

if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  lstm = tf.compat.v1.keras.layers.CuDNNLSTM
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')
  lstm =functools.partial(
      tf.keras.layers.LSTM, recurrent_activation='sigmoid')
    
    
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
     rnn(rnn_units,
         return_sequences=True,
         recurrent_initializer='glorot_uniform', # first RNN layer
         stateful=True),
    lstm(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform', # first RNN layer
        stateful=True),
     
    tf.keras.layers.Dense(vocab_size)
  ])
  return model


model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

# FOr each character, the model looks up the embedding runs the GRU 
# one timestep with the embedding as input, adn applies the dense layer 
# to generate logits predicting the log-likelihood of the next char


Is GPU available:  True


In [11]:
# ************************************************************************

### Try the model ### 

# Now run the model to see that it behaves as expected 

# First check the shape of the output: 

for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
  
print(model.summary())

(64, 100, 94) # (batch_size, sequence_length, vocab_size)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24064     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (64, None, 1024)          8396800   
_________________________________________________________________
dense (Dense)                (64, None, 94)            96350     
Total params: 12,455,518
Trainable params: 12,455,518
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# To get actual predictions from the model we need to saomple 
# from the output ditribution, to get actual character indices. 
# This distribution is defined by the logits over the vocabulary. 

# NOTE: It's important to sample from this distribution as 
# taking the argmax of the distribution can easily get the model 
# stuck in a loop. 


# Try it for the first example in the batch: 
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()


# This gives us, at each timestep, a prediction of the next character 
# index: 

print(sampled_indices) 

# Decode these to see the text predicted by this untrained model: 
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))


[76 87 40 22  8 58 34 73 26 43 66 71 87 69  0 30 91 43 35 81 63 80 86 82
 37 79 59 91  2 43 22 74 54 49 87 42 85 35 92 52 31 34 25 12 78 11  5  4
 72 90 62 79 32 61 33 18 17 20 72 20 47 20 15 17 47 66 63 23 55 23 76 74
 18 28  4 73 10 86 45 36 57 90 73 22 66 17 15 64 13 29 33 61 25 34 66 81
 45 17 78 10]
Input: 
 'getting 21 people. And they have the right to do whatever they want from an enterprise standpoint. R'

Next Char Predictions: 
 'r—K7\'_Eo;Nhm—k\nA”NFwev–xHua” N7pYT—MéF…WBE:-t,$"n“duCcD325n5R502Rhe8Z8rp3?"o)–PG]“o7h20f.@Dc:EhwP2t)'


In [13]:
# *****************************************************************

### Train the model ### 

# At this point the problem can be treated as a standard classification 
# problem. Given the previous RNN state, and the input at this time step, 
# predict the class of the next character. 


## Attach an optimizer, and a loss function 

# The standard *tf.keras.losses.sparse_softmax_crossentropy* loss 
# function works in this case because it is applied across the last 
# dimension of the predictions. 

# Because our model returns logits, we need to set the from_logits flag. 

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

# COnfigure the training procedure using the tf.leras.Model.compile method. 
# We'll use *tf.train.AdamOptimizer* with defautl arguments and 
# the loss funciton. 

model.compile(
        optimizer=tf.compat.v1.train.AdamOptimizer(), 
        loss = loss)


Prediction shape:  (64, 100, 94)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5427647


In [14]:

# ****************************************************************

### Configure checkpoints ### 

# Use a *tf.keras.callbacks.ModelCheckpoint* to ensure that checkpoints are 
# saved durinf training: 

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


# set up gpu 
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))


# **************************************************************

### Execute the training ### 

# To keep training time reasonable, use 3 epochs to train the model. 
# In Colab, set the runtime to GPU for faster training. 

EPOCHS=12

history = model.fit(dataset.repeat(), epochs=EPOCHS, 
                    steps_per_epoch=steps_per_epoch, 
                    callbacks=[checkpoint_callback])


Epoch 1/12
141/141 [==============================] - 48s 343ms/step - loss: 2.5557
Epoch 2/12
141/141 [==============================] - 46s 323ms/step - loss: 1.5492
Epoch 3/12
141/141 [==============================] - 46s 323ms/step - loss: 1.2701
Epoch 4/12
141/141 [==============================] - 46s 324ms/step - loss: 1.1560
Epoch 5/12
141/141 [==============================] - 46s 323ms/step - loss: 1.0841
Epoch 6/12
141/141 [==============================] - 46s 324ms/step - loss: 1.0302
Epoch 7/12
141/141 [==============================] - 46s 324ms/step - loss: 0.9834
Epoch 8/12
141/141 [==============================] - 46s 324ms/step - loss: 0.9385
Epoch 9/12
141/141 [==============================] - 46s 324ms/step - loss: 0.8998
Epoch 10/12
141/141 [==============================] - 46s 324ms/step - loss: 0.8584
Epoch 11/12
141/141 [==============================] - 46s 324ms/step - loss: 0.8223
Epoch 12/12
141/141 [==============================] - 46s 324ms/step - lo

In [15]:
# ****************************************************************8

### Generate text ### 

## Restore the latest checkpoint 

# To keep this prediction simple, use a batch size of 1. 

# Because of the way the RNN state is passed form timestep to 
# timestep, the model only accepts a fixed batch size once built. 

# To run the model with a different batch_size , we need to rebuild 
# the model and restore the weights from the checkpoint

tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            24064     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (1, None, 1024)           8396800   
_________________________________________________________________
dense_1 (Dense)              (1, None, 94)             96350     
Total params: 12,455,518
Trainable params: 12,455,518
Non-trainable params: 0
_________________________________________________________________


In [16]:
 
# **************************************************************

### The prediction loop ### 

# The following code block generates the text: 

    # - It starts y choosing a start string, initializing the 
    #   RNN state and setting the number of characters to generate. 
    # - Get the prediction distribution of the next character using 
    #   the start string and the RNN state, 
    # - Use a multinomial distribution to calculate the index 
    #   of the predicted character. Use this predicted character 
    #   as out next input to the model. 
    #   The RNN state returned by the model is fed back into the
    #   model so that it now has more contexxt, instead of only
    #   one word. After predicting the next word, the modified 
    #   RNN states are again fed back into the model, which is 
    #   how it learns as it gets more context from the 
    #   previously predicted words. 
    
# Looking at the generat text, you'll see that the model knows 
# when to capitalize and make paragraphs,  and it imitates a 
# Shakespear -like writing vocabulary. With the mal number of 
# training epochs, it has not yet learned to form coherent sentences. 


def generate_text(model, start_string, num_generate = 1000,
                 temperature = 1.0):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = num_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))


print(generate_text(model, start_string=u"Thank you\n", num_generate=2000, 
                    temperature=0.7))




W0728 21:18:11.101668 139847443584896 deprecation.py:323] From <ipython-input-16-c5050f58434a>:31: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


Thank you



...Well, he just responded to put America back there for loving the election.
So let me just tell you that. You’ve never spent powerful crime, and I said, "What are the thing with Hispanics who I’ve said I don’t have to say that we’re going to spend like some monster. You know, we have people that are run by people who rigged the system — we have to build a wall.
But you know what? I don’t want to do business with the evangelical support.
So you know they had this good thing.
So we have to go up to my office, and we can do very well. Some of them when them jobs. Okay? So they do is the president who wants to go and senators – and I don’t know what the hell does not sir. It’s the first time I can build the wall. One of these people are using people that work like in the world. I mean, this and I had a big number of reasons. I would have been taken away from us. I’m going to do that. Okay? I want to do a good job. It’s the way it is where right on television with with evange

In [0]:
### Advanced: Customized Training ### 

# The above trainign rpocedure is simkple. ut does not give you much control. 

# So now that you've seen how to run the model manually let'sunpack the training
# loop, and implement it ourselves. This gives a starting poont, for example,  
# to implement curriculum learning to help stabilize the model's open-loop output. 

# We will use tf.GradientTape to track the gradients. 

# This procedure works as follows: 

  # - First, initialize the RNN state. We do this by calling the
  #   tf.keras.Model.reset_sates method. 
  # - Next, iterate over the dataset(batch by batch), and calculate the 
  #   predictions associated with each. 
  # - Open a tf.GradientTape, and calculate the predictions and loss 
  #   in that context. 
  # - Calculate the gradients of the loss with respect to the model vairables
  #   using the tf.GradientTape.grads method. 
  # - Finally, take a step downwards by using the optimizer's 
  #   tf.train.Optimizer.apply_gradients method. 


In [0]:
model = build_model(
vocab_size= len(vocab), 
embedding_dim=embedding_dim, 
rnn_units=rnn_units,
batch_size=BATCH_SIZE) 

In [0]:
# optimizer = tf.train.AdamOptimizer()

# # Training step 
# EPOCHS = 7


# for epoch in range(EPOCHS):
#     start = time.time()

#     # initializing the hidden state at the start of every epoch
#     # initially hidden is None
#     hidden = model.reset_states()

#     for (batch_n, (inp, target)) in enumerate(dataset):
#           with tf.GradientTape() as tape:
#               # feeding the hidden state back into the model
#               # This is the interesting step
#               predictions = model(inp)
#               loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)

#           grads = tape.gradient(loss, model.trainable_variables)
#           optimizer.apply_gradients(zip(grads, model.trainable_variables))

#           if batch_n % 100 == 0:
#               template = 'Epoch {} Batch {} Loss {:.4f}'
#               print(template.format(epoch+1, batch_n, loss))

#     # saving (checkpoint) the model every 5 epochs
#     if (epoch + 1) % 5 == 0:
#       model.save_weights(checkpoint_prefix.format(epoch=epoch))

#     print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
#     print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

# model.save_weights(checkpoint_prefix.format(epoch=epoch))
    

In [0]:
# # Try the model again bu this time checking the gradient 

# print(generate_text(model, start_string=u"Thank you", num_generate=2000, 
#                     temp=1.0))